# Design of Kalman Filter Example

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

### Description
Let's assume we want to find the 2D position of the robot such that it's position is _inferred_ using a Kalman Filter that filters out noisy measurements and control inputs to arrive at a final estimate

The Robot has a sensor equipped that can provide its 2D position with some uncertainity

The Robot is known to be moving with constant velocity with a larger process noise in its input of velocity

### State variables
It is clear that, x and y are the _observed_ variables as they are provided by the measurement sensor

However, just having those in the state vector will not lead to good results

The velocities in x and y are the _hidden_ variables that if added to the filter will help not only in inferring the position but also the velocities itself. In other words, yes the position and velocities will be correlated

There are some _unobserved_ variables (color of robot, temperature of the wheels, etc) if added will not lead to anything useful on those variables

Hence,
Let's have $X = [x, \dot{x}, y, \dot{y}]^T$

### Order of the system
For this example, we are choosing to model the system dynamics as first order

It is tempting to model the system as second order because of the fact that velocity inputs are error prone (making us believe that perhaps second order system will track robot's position better). However, that is not the case. You are welcome to try this as an exercise and evalute the filter performance using the difference between actual state and estimated state

For first order dynamics, you can write the derivatives in discrete form (Euler method for newtonian equations)

$\dot{x} = x + \dot{x} \times \Delta t$

$\dot{y} = y + \dot{y} \times \Delta t$

Remember, that you will have to break down higher order dyamic systems to first order anyway to write the Linear State Transition equation

[This](http://www.sharetechnote.com/html/DE_HigherOrderDEtoFirstOrderDE.html) is a good example on this can be achieved

### Write the state transition equation
It helps to write equation for each component in the state vector as a function of all other state components

$x = 1x + \Delta t \dot{x} + 0 y + 0 \dot{y}$

$\dot{x} = 0x + 1 \dot{x} + 0 y + 0 \dot{y}$

$y = 0x + 0 \dot{x} + 1 y + \Delta t \dot{y}$

$\dot{y} = 0y + 0 \dot{y} + 0y + 1\dot{y}$

Hence, the state transition matrix $F$ in the equation $X = FX$ can be written as:

$\begin{bmatrix}
x \\
\dot{x} \\
y \\
\dot{y}
\end{bmatrix} = \begin{bmatrix} 1 & \Delta t & 0 & 0 \\
                                0 & 1 & 0 & 0 \\
                                0 & 0 & 1 & \Delta t \\
                                0 & 0 & 0 & 1
                \end{bmatrix} \begin{bmatrix}
x \\
\dot{x} \\
y \\
\dot{y}
\end{bmatrix}$

There can be a control input as well which might not be very apparent. Values that are typical constant can be thought of as control input. For example gravity can be a control input and so a $B$ matrix can be written and state transition matrix will take the form $X = FX + BU$

### Write the Process Noise matrix
Two things to keep in mind for Process noise matrix is: 
* The dimension of this matrix is $n \times n$, where $n$ is the dimension of the state vector
* Putting non-zero values for the components that are correlated

The actual values in the matrix itself is something that might need some tuning

For example, for the current case, it makes sense that there will be correlation between Position and velocity components

$Q = \begin{bmatrix}
0 & 0.001 & 0 & 0 \\
0.001 & 0.001 & 0 & 0 \\
0 & 0 & 0 & 0.001 \\
0 & 0 & 0.001 & 0.001
\end{bmatrix}$

For simplicity, it is often assumed that the noise matrix $Q$ is constant between consecutive updates

### Write the measurement function
Here, the main task is to determine the the matrix H that will transform the state vector to the measurement space

$Z = HX$

For our case, the $H$ will be trivial and we can guess it using dimension matching
Like so, we know that the z is 2 x 1 (there are two observed variables) so $H$ should be a 2 x 4 matrix. 
In general, $H$ is a $m \times n$ matrix where, $m$ is the number of measurements

For other cases, H can also be as simple as a matrix having conversion factors for units

Therefore,

$H = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
\end{bmatrix}$

### Write the measurement noise
Having independent and uncorrelated values for measurement is a good place to start and in practice these yield suprisingly good results especially for the problem of localization

The dimension of the matrix $R$ is $m \times m$, where $m$ is the number of observed variables within the state

For the current example, the measurement noise in X and Y are normally distributed individually and their joint distribution will have covariances has zero

Let's assume the variance in measurement is 5 $meters^2$ in each x and y

So,
$R = \begin{bmatrix}
5 & 0 \\
0 & 5 \\ 
\end{bmatrix} $ 

$R$ is the measurment noise covariance matrix

### Initial Conditions
Let's assume that the inital position $(x,y) = (0,0)$ and the velocity is also $(\dot{x}, \dot{y}) = (0,0) $

Since, this is a strong assumption, we can set the initial covariance to a large value

The Covariance matrix has the dimension of $n \times n$, where $n$ is the dimension of the state vector

$P = \begin{bmatrix}
500 & 0 & 0 & 0 \\
0 & 500 & 0 & 0 \\
0 & 0 & 500 & 0 \\
0 & 0 & 0 & 500 
\end{bmatrix}$

In [3]:
# Simulation parameters
dt = 0.01
N = 50
zs_x = np.linspace(0, 15, N)
zs_y = np.linspace(0, 5, N)

Zs = np.vstack((zs_x, zs_y))

In [4]:
# Noise matrices
q_std = 0.1
r_std = 0.35

q_std_sq = q_std ** 2
r_std_sq = r_std ** 2

Q = np.array([[0, q_std_sq, 0 , 0], [q_std_sq, q_std_sq, 0, 0], [0, 0, 0, q_std_sq], [0, 0, q_std_sq, q_std_sq]])
R = np.array([[r_std_sq, 0], [0, r_std_sq]])

In [5]:
# Initial Conditions
P = np.eye(4) * 500
X = np.zeros((4,1))

In [6]:
def state_transition(X):
    F = np.array([[1, dt, 0, 0], [0, 1, 0, 0], [0, 0, 1, dt], [0 , 0, 0, 1]])
    return F @ X

In [7]:
def measurement_function(X):
    H = np.array([[1, 0, 0, 0], [0, 0, 1, 0]])
    return H @ X

In [8]:
def predictor(X, P):
    X_pred = state_transition(X)
    F = np.array([[1, dt, 0, 0], [0, 1, 0, 0], [0, 0, 1, dt], [0 , 0, 0, 1]])
    P_pred = F @ P @ F.T + Q
    return X_pred, P_pred

In [9]:
def corrector(X, P, z):
    H = np.array([[1, 0, 0, 0], [0, 0, 1, 0]])
    K = P @ H.T @ np.linalg.inv(H @ P @ H.T + R)
    X_est = X + K @ (z - measurement_function(X))
    P_est = (np.eye(4) - K @ H) @ P
    return X_est, P_est

In [10]:
# Simulate
z = np.array([[Zs[0, 0]], [Zs[1,0]]])
z_n = z + r_std_sq * np.random.randn(2,1)
X_est, P_est = corrector(X, P, z)
Xs = [X_est]
Ps = [P_est]
Z_noisy = [z_n]
for i in range(1, N):
    z = np.array([[Zs[0, i]], [Zs[1,i]]])
    z_n = z + r_std_sq * np.random.randn(2,1)
    X_pred, P_pred = predictor(X_est, P_est)
    if i % 2 == 0:
        X_est, P_est = corrector(X_pred, P_pred, z)
    else:
        X_est = X_pred
        P_est = P_pred
        
    Xs.append(X_est)
    Ps.append(P_est)
    Z_noisy.append(z_n)

In [11]:
def draw_error_ellipse(ax, mu, sigma, sigma_bounds=[1,2,3], show_points=False, draw_arrow=False):
    w, v = np.linalg.eig(sigma)

    dom = np.linspace(-math.pi, math.pi, 200, endpoint=True)

    axes_aligned_points_bounds = dict()

    for bound in sigma_bounds:
        X = list()
        Y = list()
        a = bound * math.sqrt(w[0])
        b = bound * math.sqrt(w[1])
        for p in dom:
            X.append(a * math.cos(p))
            Y.append(b * math.sin(p))

        axes_aligned_points_bounds[bound] = np.vstack((np.array(X), np.array(Y)))

    theta = math.atan2(v[1,0], v[0,0])

    R = np.array([[math.cos(theta), -math.sin(theta)], [math.sin(theta), math.cos(theta)]])

    if show_points:
        XY = np.random.multivariate_normal(mu, sigma, 500)
        ax.scatter(XY[:,0], XY[:,1], alpha = 0.1)
    
    if draw_arrow:
        ax.arrow(0, 0, v[0,0], v[1,0], fc = 'g', width = 0.02, length_includes_head=True)
        ax.arrow(0, 0, v[0,1], v[1,1], fc='r', width = 0.02, length_includes_head=True)

    for bound, axes_aligned_points in axes_aligned_points_bounds.items():
        transformed_points = R @ axes_aligned_points + mu
        ax.scatter(transformed_points[0,:], transformed_points[1,:], c='k', s = 1, alpha = 0.5)

In [12]:
%matplotlib widget

In [13]:
# Plot Trajectory
fig, ax = plt.subplots()
X_pts = [X[0] for X in Xs]
Y_pts = [X[2] for X in Xs]

Z_n_x = [z_n[0] for z_n in Z_noisy]
Z_n_y = [z_n[1] for z_n in Z_noisy]

plt.axis('equal')
plt.grid()
plt.title("XY Position and Error vs time")
ax.plot(X_pts, Y_pts, label="Robot Trajectory")

ax.scatter(Z_n_x, Z_n_y, marker='o', c='green', label="Measurements")

i = 0
for x_i, p_i in zip(Xs, Ps):
    if i % 2 != 0:
        i = i + 1
        continue
    mu = np.array([[x_i[0,0]], [x_i[2,0]]])
    sigma = np.array([[p_i[0,0], p_i[0,2]], [p_i[2,0], p_i[2,2]]])
    draw_error_ellipse(ax, mu, sigma, sigma_bounds=[3])
    i = i + 1
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Filter Evaluation
From the computed State Covariance matrix, the bounds for std for X and Y can be obtained

These bounds represent the theoretical bounds for the performance of Kalman Filter

If the difference between actual position and estimated postion is within this bound 68% of times for $1 \sigma$ and 99.7% of times for $ 3 \sigma$, it can be said that filter is working as expected

In [14]:
one_sigma_error_y_pos = [math.sqrt(p[2,2]) for p in Ps]
one_sigma_error_y_neg = [-math.sqrt(p[2,2]) for p in Ps]

gt = [np.array([[Zs[0,i]], [Zs[1,i]]]) for i in range(Zs.shape[1])]

error = [x[2] - z[1] for x,z in zip(Xs, Z_noisy)]

x_axis = np.linspace(0, 15, len(Ps))
x_axis = x_axis.tolist()

plt.figure()
plt.grid()
plt.title('Error Bounds in 1 sigma')
plt.plot(x_axis, one_sigma_error_y_pos, '--', label='Upper bound Error')
plt.plot(x_axis, one_sigma_error_y_neg, '--', label='Lower bound Error')
plt.plot(x_axis, error, label='Error between est state and measurement')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Conclusion
1. Playing with Kalman Filter in simulation is easy. You know the $Q$ and you know the $R$
2. In practice, such evaluations with simulated as well as real data will help driving the choice of matrices
3. Experimenting with order of filter and confirming that filter doesn't diverge can be done offline or online
4. The end result is often that the Kalman filter is an inexact model of the system. This inexactness causes suboptimal behavior, which in the worst case causes the filter to diverge completely.
5. If the Kalman filter is performing optimally its estimation errors will have following properties:

   a. The mean of the estimation error is zero
   
   b. The covariance is described by Kalman Filter's covariance matrix 